In [1]:
import pandas as pd
import numpy as np
import pymysql

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

## `모델 load`

In [3]:
import os

In [4]:
os.listdir('./')

['.ipynb_checkpoints',
 '2. 연락처모델_학습(0130).ipynb',
 '2023-02-02_call_model.pt',
 'checkpoint-2500',
 'checkpoint-3000',
 'logs',
 '연락처모델_검증(0202) .ipynb',
 '연락처학습데이터(0201).csv']

In [5]:
# load model
mod = torch.load(f"2023-02-02_call_model.pt")
mod.eval()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

### `검증을 위한 method`

In [6]:
import re

class Preprocessing():

    def __init__(self, sent):
        self.sent = sent

    def clean_sentence(self):
        try:  # 무성의
            self.sent = re.sub('[^\w\s]', ' ', self.sent).strip()
            self.sent = re.sub('[.,?!ᆢ~]', ', ', self.sent)
            self.sent = re.sub('[ㄱ-ㅎ|ㅏ-ㅣ]', 'ㅋ', self.sent)
            self.sent = re.sub('니다', '니다. ', self.sent)
            self.sent = re.sub('어요', '어요. ', self.sent)
            self.sent = re.sub('\n', ' ', self.sent).strip()

        except:
            print('clean_sentence method fail')
            pass

        return self.sent

    def f_clean_sentence(self):
        try:
            self.sent = re.sub('[^\w\s]', ' ', self.sent).strip()
            self.sent = re.sub('[.,?!ᆢ~]', ', ', self.sent)
            self.sent = self.return_text(self.sent)

        except:
            print('f_clean_sentence method fail')
            pass
        return self.sent

    def sub_num(self, sent):
        hannum_list = ['일', '이', '삼', '사', '오', '육', '륙', '칠', '팔', '구', '십', '영']
        sent = re.sub(r'[0-9]', ' ', sent)

        for i in hannum_list:
            sent = re.sub(rf'{i}', '  ', sent)
        return sent

    def return_target_list(self, pattern, sent):
        word_list = []
        for i in pattern.finditer(sent):
            target_word = sent[i.start(): i.end()]
            word_list.append(target_word)
        return word_list

    def return_text(self, sent):

        nam = '[남]+'
        nyeo = '[녀]+'
        yeo = '[여]+'
        son = '[아들]+'
        ddal = '[딸]+'
        num = '[\s]*(\d){1,2}[\s]*'
        han_num = '[일이삼사오육륙칠팔구십]+'

        person_list = [nam, son, nyeo, yeo, ddal]
        num_list = [num, han_num]

        for person_pattern in person_list:
            for num_pattern in num_list:
                pattern_list = [person_pattern + num_pattern, num_pattern + person_pattern]
                target_list = []

                for pattern in pattern_list:
                    add_pattern = re.compile(pattern)
                    m = add_pattern.findall(sent)
                    if m != []:
                        target_words = self.return_target_list(add_pattern, sent)
                        target_list += target_words
                    else:
                        pass

                for i in target_list:
                    sent = sent.replace(i, self.sub_num(i))
        return sent

In [7]:
import torch.nn.functional as F

class predictModel():

    def __init__(self, model_path):
        # load model
        self.model_path = model_path
        self.model = torch.load(model_path)
        # set device
        self.device = torch.device('cpu')
        # load tokenizer
        model_name = 'beomi/KcELECTRA-base'
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def predict_sentence(self, sent):
        self.model.eval()
        Pr = Preprocessing(sent)

        if 'call' in str(self.model_path):
            sent = Pr.f_clean_sentence()

        else:
            sent = Pr.clean_sentence()

        # tokenizing
        tokenized_sent = self.tokenizer(
            sent,
            return_tensors='pt',
            truncation=True,
            add_special_tokens=True,
            max_length=512
        )
        tokenized_sent.to(self.device)

        # prediction
        with torch.no_grad():
            outputs = self.model(
                input_ids=tokenized_sent['input_ids'],
                attention_mask=tokenized_sent['attention_mask'],
                token_type_ids=tokenized_sent['token_type_ids']
            )

        # result
        per = int(str(np.array(F.softmax(outputs[0][0], dim=0).detach().cpu())[1] * 100).split('.')[0])

        if len(sent) <= 5:
            # 10자 이하는 연락처 로 탐지하지 않음
            return 1, int(per)

        result = outputs[0].detach().cpu().argmax(-1)

        if int(result) == 0:
            return int(result), int(per)

        return int(result), int(per)

### `실데이터 load`

In [9]:
result = all_mate_df.copy()
result = result[['mem_no','mate_conts', 'family_conts']]
result

,mem_no,mate_conts,family_conts
0,5,반갑습니다. 여행과 운동 그리고 음악을 좋아하는 남자입니다. 취미를 함께하며 평생...,삼남매이며 형제간에우애하고 화목한가정에서 자랐습니다.앞으로도 좋은가정을 이뤄나아가고...
1,1793293,안녕하세요 나이는 크게 중요하지 않아요 밝고 긍정적이신분 대화가 잘 통하고 의지될 ...,두분다 공무원 이시고 아버지는 교육행정직 어머니는 보건소에서 근무하십니다
2,1941692,반갑습니다 몸과정신이 모두 건강한 평범한 남자입니다 직업은 인테리어 자영업자입니다 ...,가족들은 모두 화목하게 지내고 있습니다 부모님은 건강하게 잘 지내시고 형은 결혼해서...
3,1970730,안녕하십니까? 저는 건설업.토목분야 주요 지하철터널공사.도로터널공사.수자원댐도수터널...,저의가족 3남2녀중4째입니다. 형제간에 우애가 돈독하고 년중1.2회 .형제들끼리 ...
4,2,화려한 돌씽입니다. 맘 맞는분 만나서 남은 반생 같이하고 싶습니다. 여행가는것 좋아합니다,부모님 두분 아직은 건강하시고 형제로는 남동생 하나있답니다
...,...,...,...
80056,2008527,양평에서 2남2녀중 막내로태어나 대학교 중퇴하고 음식장사하고 이건 아니다 싶어 방송...,어머니 2남2녀중 막내 형은 연락 안하고 있고 누나 구분은 다 결혼하여 잘 살고 맀어요
80057,2002670,"거짓없는속질한성격,타협할수있는배려심,긍정적사고력,재능보다노력을중시한다","무자녀,2남2녀중첫째,전원출가,부모님작고,현재는혼자생활중,남매간사이원만함"
80058,2027216,"매사에 성실하고 적당한 운동하면서 관리하고 있습니다~여행, 등산, 바다, 공연, 예...",어머니계시고 남동생은 결혼해서 잘살고있습니다~ 평범한시골집이에요~
80059,2029289,친구하고 이야기하고 서로 도우면서 살아갈 사람을 찾습니다.,누아가 서울에 있고 가끔씩 연락합니다.부모님은 중국에 계시고있습니다.


In [10]:
call_model = predictModel(model_path = f'2023-02-02_call_model.pt')

In [11]:
from tqdm import tqdm

In [12]:
for row in tqdm(range(result.shape[0])):
    text = result.loc[row, 'mate_conts']
    f_text = result.loc[row,'family_conts']
    result.loc[row,'call_result'], result.loc[row,'call_per'] = call_model.predict_sentence(str(text))
    result.loc[row,'f_call_result'], result.loc[row,'f_call_per'] = call_model.predict_sentence(str(f_text))

100%|██████████████████████████████████████████████████████████████████████████| 80061/80061 [2:30:49<00:00,  8.85it/s]


In [13]:
result

,mem_no,mate_conts,family_conts,call_result,call_per,f_call_result,f_call_per
0,5,반갑습니다. 여행과 운동 그리고 음악을 좋아하는 남자입니다. 취미를 함께하며 평생...,삼남매이며 형제간에우애하고 화목한가정에서 자랐습니다.앞으로도 좋은가정을 이뤄나아가고...,0.0,0.0,0.0,0.0
1,1793293,안녕하세요 나이는 크게 중요하지 않아요 밝고 긍정적이신분 대화가 잘 통하고 의지될 ...,두분다 공무원 이시고 아버지는 교육행정직 어머니는 보건소에서 근무하십니다,0.0,0.0,0.0,0.0
2,1941692,반갑습니다 몸과정신이 모두 건강한 평범한 남자입니다 직업은 인테리어 자영업자입니다 ...,가족들은 모두 화목하게 지내고 있습니다 부모님은 건강하게 잘 지내시고 형은 결혼해서...,0.0,0.0,0.0,0.0
3,1970730,안녕하십니까? 저는 건설업.토목분야 주요 지하철터널공사.도로터널공사.수자원댐도수터널...,저의가족 3남2녀중4째입니다. 형제간에 우애가 돈독하고 년중1.2회 .형제들끼리 ...,0.0,0.0,0.0,0.0
4,2,화려한 돌씽입니다. 맘 맞는분 만나서 남은 반생 같이하고 싶습니다. 여행가는것 좋아합니다,부모님 두분 아직은 건강하시고 형제로는 남동생 하나있답니다,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
80056,2008527,양평에서 2남2녀중 막내로태어나 대학교 중퇴하고 음식장사하고 이건 아니다 싶어 방송...,어머니 2남2녀중 막내 형은 연락 안하고 있고 누나 구분은 다 결혼하여 잘 살고 맀어요,0.0,0.0,0.0,0.0
80057,2002670,"거짓없는속질한성격,타협할수있는배려심,긍정적사고력,재능보다노력을중시한다","무자녀,2남2녀중첫째,전원출가,부모님작고,현재는혼자생활중,남매간사이원만함",0.0,0.0,0.0,0.0
80058,2027216,"매사에 성실하고 적당한 운동하면서 관리하고 있습니다~여행, 등산, 바다, 공연, 예...",어머니계시고 남동생은 결혼해서 잘살고있습니다~ 평범한시골집이에요~,0.0,0.0,0.0,0.0
80059,2029289,친구하고 이야기하고 서로 도우면서 살아갈 사람을 찾습니다.,누아가 서울에 있고 가끔씩 연락합니다.부모님은 중국에 계시고있습니다.,0.0,0.0,0.0,0.0


In [14]:
call_data = result.copy()

In [17]:
call_data.loc[call_data['call_result']==1.0].to_csv('call.csv', index=False, encoding='utf-8-sig')

In [15]:
for idx, row in call_data.loc[call_data['call_result']==1.0].iterrows():
    print(f"mem_no->{row['mem_no']}, 확률 -> {row['call_per']}, {row['mate_conts']}")

mem_no->15411, 확률 -> 98.0, 살며 사랑하며 배우며...
멋진인연 멋진만남을 바랍니다...
카톡아이디나 연락처 바랍니다...
mem_no->690153, 확률 -> 99.0, 고유 개성 및 가치관을 존중하며 경제적으로 안정된 생활을 유지하는1인입니다. 연락처 보내주시면 바로 연락드리겠습니다. Shall we ? 
mem_no->1983724, 확률 -> 99.0, 81년생 싱글 남 삼:착하게 칠:살기 공:위해 칠:그런 륙:여성과 팔:함께 칠: 삶을 팔:원함
mem_no->636664, 확률 -> 99.0, 급하지 않고 천천히 서로를 알아가면서 편하게 만나고 싶습니다 요즘 빌라건설사업을 마무리되서 조금 한가해요약속이 되면 아무때나 볼 수 있습니다 인천2호선 완정역 근처(사)한국근로장애인진흥회 기획이사 사회임대주택 건설,운영 및 공장건립기획 업무와 복지형 일자리 창출 사업을 진행하여 복지와 관련된 분과 만나고 싶습니다 010 7669 0597
mem_no->1934615, 확률 -> 73.0, 
mem_no->978955, 확률 -> 99.0, 안녕하세요! 사별한지 7년째 되어가는 워킹파더입니다 예쁜 두딸을 혼자키우고 있습니다. 둘째딸이 100일이 지날때쯤 사별해서 벌써 둘째딸이 초등학생이 됐네요 되돌아보면 시간이 빨리가네요. 지금은 자매들이다 보니 두딸이 저하고 놀아주질 않네요 ^ 거의 혼자있는 시간이 많다보니깐  많이 부족한 사람인걸 알지만 조심스럽게 이성칭구가 있으면 좋겠다는 욕심에 몇자 적어봅니다.  항상 웃음 가득한 날들만 있으시길..  부족한 저라도 좋은칭구가 필요하시면 카톡siwdd로 톡남겨주세요 ..이런 처지에 제가 먼저 말걸기가 미안해서요^.^;
mem_no->1920492, 확률 -> 86.0, 좋은 인연 만나뵙길 바랍니다 경기도 평택 거주하구여 지금은 혼자 살고 있습니다 사람 관계에 지쳐서 활발한 성격인데도 불구하고 일 외에는 밖에 나가지 않습니다 서로 마음으로 배려하며  따스한 감정으로 서로를 위해주고 아껴주는 그런 사랑 찾

In [20]:
for idx, row in call_data.loc[(call_data['call_result']==1.0) & (call_data['call_per']<95)].iterrows():
    print(f"mem_no->{row['mem_no']}, 확률 -> {row['call_per']}, {row['mate_conts']}")

mem_no->1975455, 확률 -> 93.0, 잘부탁드립니다 좋은 만남 원합니다 인연 원합니다 182/78 모두 행복하세요
mem_no->1150705, 확률 -> 94.0, 안녕하세요,  여성분께 먼저 연락 드리지 않고,  적극적이고 내외로 준비되신 40세 이하 여성분 연락 기다려봅니다.  프로필 확인해 주시고, 술,담배 다 하고, 도박,유흥 하지 않습니다. Ka톡으로 답 드릴 수 있게 센스 있게 보내주세요
mem_no->690153, 확률 -> 94.0, 고유 개성 및 가치관을 존중하며 경제적으로 안정된 생활을 유지하는1인입니다. 연락처 보내주시면 바로 연락드리겠습니다. Shall we ? 
mem_no->1983724, 확률 -> 90.0, 81년생 싱글 남 삼:착하게 칠:살기 공:위해 칠:그런 륙:여성과 팔:함께 칠: 삶을 팔:원함
mem_no->1934615, 확률 -> 79.0, 
mem_no->1885809, 확률 -> 52.0, 안녕하세요  집에 행사나 제사는없고 키180/77kg취미는 골프와 헬스.빨리걷기운등을 좋아하며 성격은 둥글둥글 털털하고 아재개그일진 몰라도 유머러스한 편이며  현제 수도귄에서 집지을 땅을 사서 집을지어 파는 건축업 대표이며 부자는 아니지만 여력이 없는편은 아니라  상대도 진심같이 노력하려는 마음만 있다면  잘살수있단 확신은 있지만 외모만 믿고 내면이 없다면  노력해도 행복할수 없다는걸 알기에  그런분은 지나가 주세요 자기소개 만으론 인연을 모르고 지나칠 수 있으니  착하고 마음고운분 계시면  톡이나  폰 연락처 남겨주세요
mem_no->1156335, 확률 -> 94.0, 그냥편한사람입니다~상대도편한 사람였음해요~^^아직도011쓰고있습니다 한결같은 사람입니당~^^
mem_no->2004693, 확률 -> 90.0, 안녕하세요 잘부탁드립니다 장난식으로만 접근하지말아주세요 연애부터시작해서 하나둘씩 잘맞으면 진지하게 결혼까지 하고싶어서 가입했습니다 연락하실생각이신거라면 JTJ611 연락주세요
mem_no->1248150,

In [18]:
call_data.loc[call_data['f_call_result']==1.0].to_csv('f_call.csv', index=False, encoding='utf-8-sig')

In [21]:
for idx, row in call_data.loc[(call_data['f_call_result']==1.0)].iterrows():
    print(f"mem_no->{row['mem_no']}, 확률 -> {row['f_call_per']}, {row['family_conts']}")

mem_no->1928725, 확률 -> 79.0, 
mem_no->1906451, 확률 -> 79.0, 
mem_no->1997956, 확률 -> 99.0, 안녕하세요 자녀는 2명이고  큰 딸은 중학생1학년이고3935 아들은 초등학생 5학년입니다.4751 그리고 1명더 있습니다. 강아지은 성별은 여자이고 나이는 7개월입니다
mem_no->1802107, 확률 -> 79.0, 
mem_no->1748159, 확률 -> 52.0, 형제  1~3 강원도 4,6 경기도  7서울  살고 있고  아이들 두명 큰 28  딸  작은 25  아들
mem_no->1910963, 확률 -> 79.0, 
mem_no->1990367, 확률 -> 79.0, 
mem_no->1952525, 확률 -> 79.0, 
mem_no->1915664, 확률 -> 79.0, 
mem_no->1936749, 확률 -> 79.0, 
mem_no->2035069, 확률 -> 79.0, 
mem_no->1911530, 확률 -> 79.0, 
mem_no->1980850, 확률 -> 79.0, 
mem_no->1816247, 확률 -> 79.0, 
mem_no->1790892, 확률 -> 79.0, 
mem_no->1712002, 확률 -> 79.0, 
mem_no->2012089, 확률 -> 79.0, 
mem_no->1991451, 확률 -> 79.0, 
mem_no->1885944, 확률 -> 79.0, 
mem_no->1522883, 확률 -> 79.0, 
mem_no->2010237, 확률 -> 79.0, 
mem_no->1799335, 확률 -> 79.0, 
mem_no->1758618, 확률 -> 79.0, 
mem_no->1446289, 확률 -> 79.0, 
mem_no->1983182, 확률 -> 79.0, 
mem_no->1773534, 확률 -> 79.0, 
mem_no->1763350, 확률 -> 78.0, 20220126  구정을 앞에두고 가입하였습니다. 제가 많이 늦었습니다.